In [1]:
from google.colab import drive  
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
%cd "/content/drive/My Drive/Datasets/REFUGE/REFUGE-Training400"
train_dir = "Training400"

/content/drive/My Drive/Datasets/REFUGE/REFUGE-Training400


In [3]:
import matplotlib.pyplot as plt
import PIL
import os
import tensorflow as tf

In [4]:
img_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255,
                                                              vertical_flip=True,
                                                              horizontal_flip=True,
                                                              rotation_range=15)


train_generator = img_datagen.flow_from_directory(train_dir, 
                                                  shuffle=True, 
                                                  batch_size=32, 
                                                  class_mode="binary",
                                                  seed=6969,
                                                  target_size=(300, 300)
                                                  )

Found 400 images belonging to 2 classes.


In [5]:
model = tf.keras.Sequential()
model.add(tf.keras.applications.InceptionResNetV2(weights = 'imagenet', input_shape=(300, 300, 3), include_top=False))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(128, activation="relu"))
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(1, activation="sigmoid"))

219070464/219055592 [==============================] - 8s 0us/step


In [6]:
model.compile(optimizer=tf.keras.optimizers.Adam(1e-3), loss="binary_crossentropy", metrics=["accuracy"])

In [7]:
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('val_accuracy') >= 0.997):
            print("\nReached 99.7% accuracy so cancelling training!")
            self.model.stop_training = True

early = tf.keras.callbacks.EarlyStopping(monitor='accuracy', patience=3, mode='max',
                                        restore_best_weights=True)


history = model.fit(train_generator,
                    steps_per_epoch = train_generator.n//train_generator.batch_size,
                    callbacks=[early],
                    epochs=10)

Epoch 1/10
12/12 [==============================] - 109s 6s/step - loss: 3.2307 - accuracy: 0.8179
Epoch 2/10
12/12 [==============================] - 51s 4s/step - loss: 1.9982 - accuracy: 0.8587
Epoch 3/10
12/12 [==============================] - 51s 4s/step - loss: 0.4941 - accuracy: 0.8913
Epoch 4/10
12/12 [==============================] - 49s 4s/step - loss: 2.1372 - accuracy: 0.8533
Epoch 5/10
12/12 [==============================] - 50s 4s/step - loss: 0.3414 - accuracy: 0.8967
Epoch 6/10
12/12 [==============================] - 51s 4s/step - loss: 1.1673 - accuracy: 0.8906
Epoch 7/10
12/12 [==============================] - 50s 4s/step - loss: 0.8590 - accuracy: 0.9022
Epoch 8/10
12/12 [==============================] - 50s 4s/step - loss: 0.3879 - accuracy: 0.9103
Epoch 9/10
12/12 [==============================] - 50s 4s/step - loss: 0.3162 - accuracy: 0.9130
Epoch 10/10
12/12 [==============================] - 50s 4s/step - loss: 0.2412 - accuracy: 0.9321


In [8]:
test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory("Test400", 
                                                  batch_size=1,
                                                  class_mode="binary",
                                                  target_size=(300, 300))

val = model.evaluate(test_generator, steps=len(test_generator))

Found 0 images belonging to 0 classes.


ValueError: ignored